We start with system commands to install the required library in the user space.

<span style="color:red"> If this kernel is hosted locally only run this once and restart the kernel afterwords! </span>

In [ ]:
import sys
!{sys.executable} -m pip install pandas --user
!{sys.executable} -m pip install matplotlib --user
!{sys.executable} -m pip install scipy --user
!{sys.executable} -m pip install bokeh --user

# <span style="color:green">Start Here</span>

In [43]:
import pandas as pd
import numpy as np
import scipy.stats as st
import collections
import numbers
from IPython.display import display,HTML
from bokeh.plotting import figure, output_notebook, show
from functools import reduce

#for if I end up using bokeh
output_notebook()

Loading BokehJS ...

Defines a data structure to hold all the relevant information for a option at a given time

In [44]:
class OptionData:
    def __init__(self,S,δ,r,σ,K,T):
        self.S = S
        self.δ = δ
        self.r = r
        self.σ = σ
        self.K = K
        self.T = T
        
        #renamed to fit how these formulas are normally expressed
        N = st.norm.cdf
        ϕ = st.norm.pdf
        
        d1 = (np.log(S/K)+(r-δ+0.5*np.power(σ,2))*T)/(σ*np.sqrt(T))
        d2 = (np.log(S/K)+(r-δ-0.5*np.power(σ,2))*T)/(σ*np.sqrt(T))
        self.d1 = d1
        self.d2 = d2
        
        self.C_Eur = S*np.exp(-δ*T)*N(d1)-K*np.exp(-r*T)*N(d2)
        self.P_Eur = K*np.exp(-r*T)*N(-d2)-S*np.exp(-δ*T)*N(-d1)
        
        #European greeks
        self.Δ_Call=np.exp(-δ*T)*N(d1)
        self.Δ_Put=-np.exp(-δ*T)*N(-d1)
        
        self.Γ_Call=(1.0/(S*self.σ*np.sqrt(T)))*np.exp(-δ*T)*ϕ(d1)
        self.Γ_Put=self.Γ_Call
        
        #Scaled by 1/100
        self.ν_Call=(S*np.exp(-δ*T)*ϕ(d1)*np.sqrt(T))/100.0
        self.ν_Put=self.ν_Call
        
        #Scaled by 1/365
        self.Θ_Call=(δ*S*np.exp(-δ*T)*N(d1)-r*K*np.exp(-r*T)*N(d2)-(σ/(2*np.sqrt(T)))*S*np.exp(-δ*T)*ϕ(d1))/365.0
        self.Θ_Put=(r*K*np.exp(-r*T)*N(-d2)-δ*S*np.exp(-δ*T)*N(-d1)-(self.σ/(2*np.sqrt(T)))*S*np.exp(-δ*T)*ϕ(d1))/365.0
        
        #Scaled 1/10,000
        self.ρ_Call=(T*K*np.exp(-r*T)*N(d2))/10000.0
        self.ρ_Put=(-T*K*np.exp(-r*T)*N(-d2))/10000.0
        
        #Scaled 1/10,000
        self.Ψ_Call=(-T*S*np.exp(-δ*T)*N(d1))/10000.0
        self.Ψ_Put=(T*S*np.exp(-δ*T)*N(-d1))/10000.0
        
    def geometricAsianPrices(self,n):
        S = self.S
        δ = self.δ
        r = self.r
        σ = self.σ
        K = self.K
        T = self.T
        
        σ_ = (self.σ/n)*np.sqrt((n+1)*(2*n+1)/6)
        σ2 = np.power(self.σ,2)
        n2 = np.power(n,2)
        δ_ = 0.5*(self.r*((n-1)/n) + (self.δ + 0.5*σ2)*((n+1)/n) - (σ2/n2)*((n+1)*(2*n+1)/6))
        
        N = st.norm.cdf
        
        d1 = (np.log(S/K)+(r-δ_+0.5*np.power(σ_,2))*T)/(σ_*np.sqrt(T))
        d2 = (np.log(S/K)+(r-δ_-0.5*np.power(σ_,2))*T)/(σ_*np.sqrt(T))
        
        C = S*np.exp(-δ_*T)*N(d1)-K*np.exp(-r*T)*N(d2)
        P = K*np.exp(-r*T)*N(-d2)-S*np.exp(-δ_*T)*N(-d1)
        
        return C,P,d1,d2,σ_,δ_


def toFrame(obj):
    return pd.DataFrame.from_dict({k:v if hasattr(v, '__iter__') else [v] for k,v in obj.__dict__.items()})
        

In [45]:
ourData = OptionData(S = 105.99, K = 105, σ = 0.3128, r = 0.0017, T = 351/365, δ = 0.017)

In [46]:
toFrame(ourData)

,S,δ,r,σ,K,T,d1,d2,C_Eur,P_Eur,...,Γ_Call,Γ_Put,ν_Call,ν_Put,Θ_Call,Θ_Put,ρ_Call,ρ_Put,Ψ_Call,Ψ_Put
0,105.99,0.017,0.0017,0.3128,105,0.961644,0.135999,-0.170743,12.467405,13.024525,...,0.011961,0.011961,0.404172,0.404172,-0.015529,-0.019898,0.004357,-0.005724,-0.005556,0.004471


# Monte Carlo Simulation

In [47]:
#normal and poission random variables
from scipy.stats import norm, poisson


class PoissonData:
    
    def __init__(self,α_j,σ_j,λ):
        self.α_j = α_j
        self.σ_j = σ_j
        self.λ = λ
        
#returns an iterator of trials allowing for laziness to discard uneeded data
def runMonteCarlo(iterations,trials,stockData,exerciseFun,antitheticVariate=True,poissonData=None):
    r, δ, σ ,T = stockData.r, stockData.δ, stockData.σ ,stockData.T
    h = T/iterations
    rootH = np.sqrt(h)
    odd = True
    while trials>0:
        if antitheticVariate and not odd:
            norms = -norms
            trials -= 1
        else:
            norms = norm.rvs(size=iterations)
        odd = not odd
        
        S = stockData.S    
        prices = np.copy(norms)
        prices[0] = S
        for i in range(1,iterations):
            jump=1
            α = r
            if poissonData:
                λh = poissonData.λ*T/iterations
                m = poisson.rvs(λh)
                ws = norm.rvs(size=m)
                α_j = poissonData.α_j
                σ_j = poissonData.σ_j
                α -= poissonData.λ*T*(np.exp(α_j) - 1)
                σ_j2 = np.power(σ_j,2)
                jump = np.exp(m*(α_j - 0.5*σ_j2) + σ_j*(np.sum(ws)))
            Snext = S*np.exp((α - δ - 0.5*np.power(σ,2))*h + σ*rootH*norms[i])*jump
            prices[i] = Snext
            S=Snext
        
        call,put = exerciseFun(prices,stockData)
        
        yield norms,prices,call,put
        
def monteCarloToDataFrame(monteCarlo):
    l = []
    for norms,prices,call,put in monteCarlo:
        d = {"call payoff":call, "put payoff":put}
        d["iterations"] = len(prices)
        for i in range(0,len(prices)):
            d["S_"+str(i)] = prices[i]
            d["z_"+str(i)] = norms[i]
        l.append(d)
    return pd.DataFrame(data = l)

def getMonteCarloFrameStats(frame):
    dTemp =  {}
    dTemp["call"] = np.exp(-ourData.r*ourData.T)*np.mean(frame["call payoff"])
    dTemp["put"] = np.exp(-ourData.r*ourData.T)*np.mean(frame["put payoff"])
    dTemp["call σ"] = np.sqrt(np.var(frame["call payoff"]))
    dTemp["put σ"] = np.sqrt(np.var(frame["put payoff"]))
    
    dTemp["call σ normalized"] = dTemp["call σ"]/np.sqrt(len(frame["call payoff"]))
    dTemp["put σ normalized"] = dTemp["put σ"]/np.sqrt(len(frame["put payoff"]))
    for i in range(0, frame["iterations"][0]):
        dTemp["sample σ "+str(i)] = np.sqrt(np.var(frame["z_"+str(i)]))
        dTemp["sample mean "+str(i)] = np.mean(frame["z_"+str(i)])
        dTemp["price σ"+str(i)] = np.sqrt(np.var(frame["S_"+str(i)]))
        dTemp["price mean "+str(i)] = np.mean(frame["S_"+str(i)])
    return pd.DataFrame(data=[dTemp])

# Exercise Functions

In [48]:
def europeanExerciseFunction(prices,stockData) :
    call = max(prices[-1]-stockData.K,0)
    put = max(stockData.K-prices[-1],0)
    return call,put
    
def asianArithmeticExerciseFunction(prices,stockData) :
    arithMean = np.mean(prices)
    call = max(arithMean - stockData.K,0)
    put = max(stockData.K - arithMean,0)
    return call,put

def asianGeometricExerciseFunction(prices,stockData) :
    geoMean = reduce(lambda k,p: np.power(p,1/len(prices))*k, prices, stockData.K)
    call = max(geoMean - stockData.K,0)
    put = max(stockData.K - geoMean,0)
    return call,put


In [49]:
frame = monteCarloToDataFrame(runMonteCarlo(1,10000,ourData,europeanExerciseFunction,antitheticVariate=True))

In [50]:
frame

,call payoff,put payoff,iterations,S_0,z_0
0,0.99,0,1,105.99,0.030518
1,0.99,0,1,105.99,-0.030518
2,0.99,0,1,105.99,-0.637868
3,0.99,0,1,105.99,0.637868
4,0.99,0,1,105.99,1.256324
...,...,...,...,...,...
19995,0.99,0,1,105.99,0.959575
19996,0.99,0,1,105.99,0.500133
19997,0.99,0,1,105.99,-0.500133
19998,0.99,0,1,105.99,-0.209690


Here is how you look at a row of the data frame

In [51]:
start = 1000
end = 1000
frame.loc[start:end]

,call payoff,put payoff,iterations,S_0,z_0
1000,0.99,0,1,105.99,0.5865


Heres how you look at a cell

In [52]:
row = 2000
column = "S_0"
frame[column][row]

105.99

Or alternatively 

In [53]:
startRow = 2000
endRow = 2000
column = "S_0"
frame.loc[startRow:endRow,"S_0"]

2000    105.99
Name: S_0, dtype: float64

# Results of Simulation and Related Statistical Information

In [54]:
stats = getMonteCarloFrameStats(frame)

In [55]:
stats

,call,put,call σ,put σ,call σ normalized,put σ normalized,sample σ 0,sample mean 0,price σ0,price mean 0
0,0.988383,0.0,0.0,0.0,0.0,0.0,1.003085,0.0,0.0,105.99


# Black Scholes Geometric Asian Option Pricing

In [56]:
C_asian,P_asian,d1_asian,d2_asian,σ_asian,δ_asian = ourData.geometricAsianPrices(24)

In [57]:
pd.DataFrame(data = [{
    "C":C_asian,
    "P":P_asian,
    "d1*":d1_asian,
    "d2*":d2_asian,
    "σ*":σ_asian,
    "δ*":δ_asian
}])

,C,P,d1*,d2*,σ*,δ*
0,7.288739,7.926871,0.057876,-0.124746,0.186229,0.017808


# Monte Carlo for Arithmatic Mean Asian with Poisson Jumps

In [58]:
poissonData = PoissonData(α_j = 0.0726,σ_j = 0.028,λ = 2)
iterations = 24

In [59]:
frame1 = monteCarloToDataFrame(runMonteCarlo(iterations,1000,ourData,asianArithmeticExerciseFunction,antitheticVariate=True,poissonData=None))
frame2 = monteCarloToDataFrame(runMonteCarlo(iterations,1000,ourData,asianArithmeticExerciseFunction,antitheticVariate=True,poissonData=poissonData))

In [60]:
from bokeh.palettes import viridis

graph = 256

p = figure(title="Monte Carlo Asset Price Simulation : 1st "+ str(graph) +" Paths",plot_width=600, plot_height=400)

xvalues = list(range(0,iterations))

yvalues = [[frame2["S_"+str(j)][i*2] for j in xvalues] for i in range(0,graph)]

p.multi_line( [xvalues]*graph,  yvalues , line_width=4, color = viridis(graph))

show(p)

In [61]:
hist,_ = np.histogram(frame1["S_23"],bins=50,range=(0,300))

p = figure(title="Monte Carlo Asset Price Simulation : 1st 25 Paths",plot_width=600, plot_height=400)

x = list(range(3,300,6))
print(len(x))
print(len(hist))
p.vbar(x=x, width=2, bottom=0,
       top=hist, color=viridis(50))

show(p)

50
50


In [62]:
hist,_ = np.histogram(frame2["S_23"],bins=50,range=(0,300))

p = figure(title="Monte Carlo Asset Price Simulation : 1st 25 Paths",plot_width=600, plot_height=400)

x = list(range(3,300,6))
print(len(x))
print(len(hist))
p.vbar(x=x, width=2, bottom=0,
       top=hist, color=viridis(50))

show(p)

50
50


In [67]:
stats1 = getMonteCarloFrameStats(frame2)

In [68]:
stats1

,call,put,call σ,put σ,call σ normalized,put σ normalized,sample σ 0,sample mean 0,price σ0,price mean 0,...,price σ21,price mean 21,sample σ 22,sample mean 22,price σ22,price mean 22,sample σ 23,sample mean 23,price σ23,price mean 23
0,7.920054,7.423878,13.40591,9.702355,0.299765,0.216951,0.97429,0.0,1.421085e-14,105.99,...,33.750021,105.234972,0.993432,0.0,34.175354,105.058379,0.97194,0.0,35.003149,104.994829


In [66]:
frame3 = monteCarloToDataFrame(runMonteCarlo(iterations,1000,ourData,asianGeometricExerciseFunction,antitheticVariate=True,poissonData=poissonData))
stats2 = getMonteCarloFrameStats(frame2)
stats2

,call,put,call σ,put σ,call σ normalized,put σ normalized,sample σ 0,sample mean 0,price σ0,price mean 0,...,price σ21,price mean 21,sample σ 22,sample mean 22,price σ22,price mean 22,sample σ 23,sample mean 23,price σ23,price mean 23
0,7.920054,7.423878,13.40591,9.702355,0.299765,0.216951,0.97429,0.0,1.421085e-14,105.99,...,33.750021,105.234972,0.993432,0.0,34.175354,105.058379,0.97194,0.0,35.003149,104.994829


In [70]:
β = 1.0202
c_arith_adj = stats1["call"][0] + β*(C_asian-stats2["call"][0])
p_arith_adj = stats1["put"][0] + β*(P_asian-stats2["put"][0])
(c_arith_adj,p_arith_adj)

(7.275986192346686, 7.9370314680349)